In [1]:
# !pip3 install pandas
# !pip3 install numpy
# !pip3 install Keras
# !pip3 install keras-bert
# !pip3 install tqdm

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#!pip install Keras
#!pip install keras-bert

In [5]:
import tensorflow as tf
import keras as keras
import keras.backend as K
from keras.models import load_model

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

import pandas as pd
import numpy as np
from tqdm import tqdm
import gc

Using TensorFlow backend.


# Read The data

In [6]:
data = pd.read_csv("keyMssg_train.csv")

In [7]:
data.head()

,Text,Label
0,Over 10 00 patients have been treated with XAL...,0
1,"Female , 65 years of age Theresa is not an act...",8
2,Discontinue ELIQUIS and begin taking the new a...,3
3,- 3 + t Examples of locations of injury includ...,13
4,Alphagan Brimonidine,0


In [8]:
data.iloc[16]

Text     * An attempt should be made to locate missing ...
Label                                                    8
Name: 16, dtype: object

In [9]:
from keras.utils import np_utils

# Build the model

In [10]:
SEQ_LEN = 60
BATCH_SIZE = 18
EPOCHS = 3
LR = 4e-5

pretrained_path = '../BERT_BioBERT/Text_semantic/model_folder/'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = 'Text'
LABEL_COLUMN = 'Label'

In [11]:
!ls config_path

ls: cannot access 'config_path': No such file or directory


### Get Tokenizer

In [12]:
token_dict = load_vocabulary(vocab_path)
tokenizer = Tokenizer(token_dict)

### Load and Convert to data that BERT understand

In [13]:
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    np.random.shuffle(items)
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

In [14]:
def load_data(data_df):
    #data_df = pd.read_csv(path, nrows=10000)
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

In [15]:
#train_y

In [16]:
from keras.utils import to_categorical

In [17]:
train_x, train_y = load_data(data)
gc.collect()

100%|██████████| 13031/13031 [00:09<00:00, 1446.05it/s]


0

In [18]:
train_y = to_categorical(train_y)

In [19]:
train_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### Load Model from Checkpoint

In [20]:
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,
)

W0109 09:36:11.071736 139821173102400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0109 09:36:11.091523 139821173102400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0109 09:36:11.142937 139821173102400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0109 09:36:11.144199 139821173102400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0109 09:36:11.152195 

### Create Keras Model and compile it

In [21]:
inputs = model.inputs[:2]
dense = model.layers[-3].output
outputs = keras.layers.Dense(16, activation='softmax', kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                             name = 'real_output')(dense)

decay_steps, warmup_steps = calc_train_steps(
    train_y.shape[0],
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
)

model = keras.models.Model(inputs, outputs)
model.compile(
    AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

W0109 09:36:32.531810 139821173102400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [22]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 60)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 60)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 60, 768), (3 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 60, 768)      1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [23]:
model.fit(
        train_x,
        train_y,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2
    )

W0109 09:36:32.728948 139821173102400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10424 samples, validate on 2607 samples
Epoch 1/3
10424/10424 [==============================] - 359s 34ms/step - loss: 1.3850 - acc: 0.6089 - val_loss: 0.8362 - val_acc: 0.7603
Epoch 2/3
10424/10424 [==============================] - 343s 33ms/step - loss: 0.6251 - acc: 0.8291 - val_loss: 0.7443 - val_acc: 0.7944
Epoch 3/3
10424/10424 [==============================] - 343s 33ms/step - loss: 0.3562 - acc: 0.9064 - val_loss: 0.7468 - val_acc: 0.7979


### Saving the model

In [21]:
from keras_bert import get_custom_objects

In [ ]:
# from keras.models import load_model

# model.save('model_finalized_v1_81_71.h5')

In [ ]:
# trained_model.save_weights('my_awesome_model') # save it
# model = load_model('my_awesome_model', encoder_model) # load it later and use it!

# Prediction script for test file

# Final Prediction Script

In [27]:
#!pip install tensorflow

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import get_custom_objects
import keras
import time

SEQ_LEN = 60

In [2]:
#load model one time
def loadModel_vocab(path = "/home/ajeetsingh/Documents/VCC/keyMessage/BERT"):
    vocab_path = path+'/vocab_uncased_81_71.txt'
    token_dict = load_vocabulary(vocab_path)
    tokenizer = Tokenizer(token_dict)
    
    model = keras.models.load_model(path +'/model_finalized_v1_81_71.h5', custom_objects=get_custom_objects())
    return model, tokenizer
    

In [10]:
#get prediction from loaded model
def getPredictionFromBERT(text, model, tokenizer):
    
    ids, segments = tokenizer.encode(input_text, max_len=SEQ_LEN)
    model_input = [np.array([ids]), np.zeros_like([ids])]
    prediction = model1.predict(model_input)
    
    return prediction    

In [3]:
#main function to load and get final prediction
start_time = time.time()
model1, tokenizer1 = loadModel_vocab()
print("--- %s seconds ---" % (time.time() - start_time))

W0826 08:48:40.697624 140524488652608 deprecation_wrapper.py:119] From /home/ajeetsingh/Documents/VCC/keyMessage/keyMessageBERT/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 08:48:40.734422 140524488652608 deprecation_wrapper.py:119] From /home/ajeetsingh/Documents/VCC/keyMessage/keyMessageBERT/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 08:48:40.802501 140524488652608 deprecation_wrapper.py:119] From /home/ajeetsingh/Documents/VCC/keyMessage/keyMessageBERT/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 08:48:40.803157 140524488652608 deprecation_wrapper.py:119] From /home/ajeetsingh/Documents/VCC/keyMessage/keyMessageBERT/lib/python3.6/site

--- 28.512789011001587 seconds ---


In [30]:
input_text = "For patients who remain symptomatic on a short acting bronchodilator"

In [32]:
start_time = time.time()
print(getPredictionFromBERT(input_text, model1, tokenizer1))
print("--- %s seconds ---" % (time.time() - start_time))

[[2.5454251e-04 2.7663815e-03 2.8993129e-03 1.5515804e-02 6.6051739e-03
  6.3632051e-03 8.8909721e-01 1.9974783e-03 4.3881793e-02 1.1408570e-03
  1.2701717e-03 1.8673387e-03 1.6779760e-02 8.8643614e-04 6.7116898e-03
  1.9628755e-03]]
--- 0.13864898681640625 seconds ---


## prediction testing for batch of records

In [42]:
import tensorflow as tf
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import get_custom_objects
import keras as keras
import os
import numpy as np
import pandas as pd

SEQ_LEN = 60

DATA_COLUMN = 'Text'

def convert_test(test_df, tokenizer):
    #global tokenizer
    indices = []
    for i in range(len(test_df)):
        ids, segments = tokenizer.encode(test_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]





def getPredictionFromBERTForBatchOfRecords(df,model, tokenizer):
    text_x = convert_test(df, tokenizer1)
    result = model.predict(test_x)
    return result
    

In [43]:
#load and get prediction
def load_test(path):
    data_df = pd.read_csv(path, nrows=100)
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    #data_x = convert_test(data_df)
    return data_df

test_x = load_test('/home/ajeetsingh/Documents/VCC/keyMessage/BioBert/train.csv')


In [12]:
# start_time = time.time()
# print(getPredictionFromBERTForBatchOfRecords(test_x, model1, tokenizer1))
# print("--- %s seconds ---" % (time.time() - start_time))

# Prediction Script for a batch of records

In [94]:
data = pd.read_csv("/home/ajeeth/Documents/VCC/keyMessage/Indication_model/merk_RTE_slices.csv")

In [98]:
data1 = pd.read_csv("/home/ajeeth/Documents/VCC/keyMessage/Indication_model/RTE_slices_35.csv")

In [99]:
data.shape

(2212, 86)

In [100]:
data1.shape

(1501, 86)

In [101]:
data2 = pd.concat([data, data1])

In [102]:
data['text'].isnull().sum()

68

In [103]:
data2.dropna(subset=['text'], inplace=True)

In [104]:
data2.reset_index(inplace=True)

In [105]:
data2.shape

(3599, 87)

In [106]:
3713-114

3599

In [107]:
data2.to_csv("/home/ajeeth/Documents/VCC/keyMessage/Indication_model/Merck_RTE_Slices_Indication.csv", index = False)

In [108]:
data2.shape

(3599, 87)

In [4]:
for i in range(data.shape[0]):
    data['text'][i] = str(data['text'][i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
data.head()

,index,slice_link,slide_link,text
0,0,http://c-cube-dev.indegene.com:8080/static/get...,http://c-cube-dev.indegene.com:8080/static/get...,FORXIGAⓇy XIGDUO® XR: the most prescribed iSGL...
1,1,http://c-cube-dev.indegene.com:8080/static/get...,http://c-cube-dev.indegene.com:8080/static/get...,LESS GLUCOSE. MORE RESULTS.
2,2,http://c-cube-dev.indegene.com:8080/static/get...,http://c-cube-dev.indegene.com:8080/static/get...,Significant reductions in HbA1c 2 * Decrease o...
3,3,http://c-cube-dev.indegene.com:8080/static/get...,http://c-cube-dev.indegene.com:8080/static/get...,ONCE A xigduo day. XR dapagliflozin + extended...
4,4,http://c-cube-dev.indegene.com:8080/static/get...,http://c-cube-dev.indegene.com:8080/static/get...,See the local product feature summary (CPR) fo...


In [6]:
import tensorflow as tf
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import get_custom_objects
import keras as keras
import os
import numpy as np
import pandas as pd
import time

SEQ_LEN = 60

In [7]:
# /home/ajeeth/Documents/VCC/gitVCC/vcc-models/Ajeet_4616/KeyMessage_BERT_81_2/model

DATA_COLUMN = 'text'
vocab_path = '../model/keyMessage_vocab_uncased_81_71.txt'

token_dict = load_vocabulary(vocab_path)
tokenizer1 = Tokenizer(token_dict)

print("Model Loading....")
start_time = time.time()
loaded_model = keras.models.load_model('../model/keyMessage_model_finalized_v1_81_71.h5', custom_objects=get_custom_objects())
print("--- %s seconds ---" % (time.time() - start_time))

print("Model Loaded!!")

print("Input file is loading")

def convert_test(test_df, tokenizer):
    #global tokenizer
    indices = []
    for i in range(len(test_df)):
        ids, segments = tokenizer.encode(test_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def load_test(path):
    data_df = pd.read_csv(path)
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = convert_test(data_df, tokenizer1)
    return data_x


test_x = load_test('/home/ajeeth/Documents/VCC/keyMessage/Indication_model/Merck_RTE_Slices_Indication.csv')

print("Input file loaded")

start_time = time.time()

result = loaded_model.predict(test_x)

print("--- %s seconds ---" % (time.time() - start_time))

# print("Saving into Numpy")

# np.save('spanish_numpy_array', result) 

#Get the index of the max prob values

print("Getting Prediction")
preds = []
for prediction in result:
    preds.append(np.argmax(prediction))

#key message category list
le_classes = ['Brand Information',
'Comparative Information',
'Cost',
'Disease Management',
'Dosing & Administration',
'Efficacy',
'Indication',
'Pathology',
'Patient Profile',
'Pharmacology',
'Physician Enablement',
'Risk Factor',
'Safety & Tolerability',
'Study Design',
'Summary',
'Unmet Need']

#get the key message category name from the index
pred_class = []

for i in preds:
    pred_class.append(le_classes[i])


#append the prediction with the original dataframe
test_x = pd.read_csv('/home/ajeeth/Documents/VCC/keyMessage/Indication_model/Merck_RTE_Slices_Indication.csv')
test_x['Prediction_iDA_BERT_KeyMessage'] = pred_class

# #save the data file

# test_x.to_csv("Spanish_5200_BERT_Prediction.csv", index = False)




# print("Prediction Completed")





W0116 17:14:47.711898 139746573064000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0116 17:14:47.733593 139746573064000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0116 17:14:47.768815 139746573064000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0116 17:14:47.769380 139746573064000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0116 17:14:47.774981

Model Loading....


W0116 17:14:52.390303 139746573064000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0116 17:14:52.482115 139746573064000 deprecation.py:323] From /home/ajeeth/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


--- 20.43277072906494 seconds ---
Model Loaded!!
Input file is loading
Input file loaded
--- 166.29082012176514 seconds ---
Getting Prediction


In [8]:
test_x.head(2)

,index,S. No.,Asset Name,Asset Type,Slide Name,deckid,slideid,_id,Modified Asset ID,Modified Slide ID,...,messagecategories.Resources,messagecategories.Risk Markers,messagecategories.Safety & Tolerability,messagecategories.Summary,messagecategories.Trial Design,messagecategories.Unmet Needs,corrected.mediatype,corrected.messagecategories,corrected.semantictype,Prediction_iDA_BERT_KeyMessage
0,0,NaN,NaN,NaN,NaN,ObjectId(5daec67f6640611af0f00197),ObjectId(5daec6816640611af0f00198),ObjectId(5daec6a06640611af0f00199),NaN,NaN,...,0.1075,NaN,0.0173,0.0041,NaN,0.0046,"{""Valid Mailbox"":1.0}",{},{},Physician Enablement
1,1,NaN,NaN,NaN,NaN,ObjectId(5daec67f6640611af0f00197),ObjectId(5daec6816640611af0f00198),ObjectId(5daec6a26640611af0f0019a),NaN,NaN,...,0.0096,NaN,0.0144,0.0035,NaN,0.0053,{},"{""NA"":1.0}",{},Patient Profile


In [9]:
base_url = "http://c-cube-dev.indegene.com/asset/"
test_x['deckid'] = test_x.deckid.str.replace('ObjectId','')
test_x['deckid'] = test_x.deckid.str.replace('(','')
test_x['deckid'] = test_x.deckid.str.replace(')','')

test_x['_id'] = test_x["_id"].str.replace('ObjectId','')
test_x['_id'] = test_x["_id"].str.replace('(','')
test_x['_id'] = test_x["_id"].str.replace(')','')

test_x["slice_link"]= base_url + test_x["deckid"] +"/slices/" + test_x["_id"] +".png"


In [10]:
test_x.to_csv("/home/ajeeth/Documents/VCC/keyMessage/Indication_model/Merck_RTE_Slices_Indication_predicted_BERT.csv", index = False)

In [27]:
test_x[test_x['Prediction_BERT'] != test_x['Prediction_BERT_test']]

,text,Label,Prediction_BERT,Prediction_BERT_test


In [28]:
actual_class = []

for i in test_x['Label']:
#     print(le_classes[i])
    actual_class.append(le_classes[i])
#     break

In [29]:
test_x['Actual_class'] = actual_class

In [30]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(test_x['Actual_class'], test_x['Prediction_BERT']))

                         precision    recall  f1-score   support

      Brand Information       0.93      0.95      0.94       129
Comparative Information       0.88      0.44      0.58        16
                   Cost       0.94      0.93      0.94       127
     Disease Management       0.86      0.91      0.89       170
Dosing & Administration       0.93      0.95      0.94       288
               Efficacy       0.91      0.92      0.91       412
             Indication       0.92      0.94      0.93       128
              Pathology       0.81      0.86      0.83       134
        Patient Profile       0.96      0.87      0.91       179
           Pharmacology       0.95      0.93      0.94       167
   Physician Enablement       0.97      0.97      0.97       234
            Risk Factor       0.61      0.69      0.65        16
  Safety & Tolerability       0.95      0.96      0.96       805
           Study Design       0.91      0.94      0.92       420
                Summary 

In [21]:
merk_id = ["5db9922666406147260e211e","5dc2942d66406165645cf4a8","5dc2a1266640610638b02c90","5dc2a1406640610638b02ca4","5db97c1866406147260e202f","5dc25c9e66406165645cf31c","5dc25cc466406165645cf31e","5dc25cf966406165645cf32d","5dc25d1366406165645cf33d","5dc3f02866406111e5c388bf","5dbffa1d66406147260e2b15","5dc3b16d66406111e5c38879","5dc3f01066406111e5c388bd","5dc27d8166406165645cf438","5dc2976366406165645cf4f3","5dc2977a66406165645cf4f5","5dc2a08d6640610638b02c7d","5dc2a09f6640610638b02c7f","5dc2a0b06640610638b02c81","5dc2a0c36640610638b02c84","5dbabb0c66406147260e28bf","5dc27d1c66406165645cf436","5dc1230666406147260e2b6a","5dbae29d66406147260e2945","5dbfd66166406147260e2a25","5dbfd58866406147260e29ea","5db96b6e66406147260e1f53","5dbfcd0b66406147260e299a","5db97c9366406147260e2070","5db97c5366406147260e2047","5dc25d3766406165645cf362","5db9916466406147260e2106","5daf37f66640611af0f004c6","5daec67f6640611af0f00197","5db9905b66406147260e20c9","5dc25d6266406165645cf38f","5dc2c08666406111e5c38803","5dbfd15b66406147260e29c1","5db97be566406147260e201e","5db990e666406147260e20e4","5dc3ba5066406111e5c388ac","5dc3ba7966406111e5c388b3"]

In [22]:
dev_id = ["5d11cf7a3f6c5432268f7ae4","5d11cebc3f6c5432268f7a8b","5d11ce583f6c5432268f7a58","5d11cdbd3f6c5432268f79ff","5d11cd9e3f6c5432268f79e6","5d11e1f83f6c5432268f7bca","5d11e1493f6c5432268f7ba7","5d11e04f3f6c5432268f7b79","5d11dfb63f6c5432268f7b55","5d11df293f6c5432268f7b2e","5d11e2cf3f6c5432268f7bfa","5d11e35d3f6c5432268f7c2c","5d11e3e33f6c5432268f7c5b","5d11e7683f6c5432268f7cab","5d22dd893f6c54102196c807","5d22de2b3f6c54102196c823","5d22de9a3f6c54102196c861","5d22df0e3f6c54102196c891","5d22df8f3f6c54102196c906","5d22edbf3f6c54102196ca9e","5d22ede03f6c54102196caac","5d22ee6e3f6c54102196cae7","5d22ef323f6c54102196cb9d","5d22efa83f6c54102196cc3e","5d2318673f6c54102196cca0"]

In [5]:
slide_merk = pd.read_csv("/home/ajeeth/Documents/VCC/keyMessage/Indication_model/med_comp/slides_merk_rte.csv")

In [6]:
slide_35 = pd.read_csv("/home/ajeeth/Documents/VCC/keyMessage/Indication_model/med_comp/slides_merk_35.csv")

In [7]:
data_rte = pd.read_csv("/home/ajeeth/Documents/VCC/keyMessage/Indication_model/med_comp/Merck_RTE_Slices_Indication_predicted.csv")

In [9]:
slide_35['deckid'][0]

'ObjectId(5d11cd9e3f6c5432268f79e6)'

In [11]:
data_rte['deckid'][1]

'5daf37f66640611af0f004c6'

In [14]:
slide_35['deckid'] = slide_35["deckid"].str.replace('ObjectId','')
slide_35['deckid'] = slide_35["deckid"].str.replace('(','')
slide_35['deckid'] = slide_35["deckid"].str.replace(')','')


In [16]:
slide_merk['deckid'] = slide_merk["deckid"].str.replace('ObjectId','')
slide_merk['deckid'] = slide_merk["deckid"].str.replace('(','')
slide_merk['deckid'] = slide_merk["deckid"].str.replace(')','')


In [20]:
mapping_35 = dict(zip(slide_35['deckid'], slide_35['filename']))
slide_merk = dict(zip(slide_merk['deckid'], slide_merk['filename']))

In [17]:
slide_merk['deckid'][0]

'5daec67f6640611af0f00197'

In [27]:
base_url_merk = "https://icb-merck.indegene.com/asset/"
base_url_dev = "http://c-cube-dev.indegene.com/asset/"
base_url_merk+data_rte["deckid"][10]+"/"+slide_merk[data_rte['deckid'][10]]

'https://icb-merck.indegene.com/asset/5db97c1866406147260e202f/HEDIS_HQ_Email_1.png'

In [31]:
slide_link_list = []
base_url_merk = "https://icb-merck.indegene.com/asset/"
base_url_dev = "http://c-cube-dev.indegene.com/asset/"

for i in range(data_rte.shape[0]):
    if(data_rte['deckid'][i] in merk_id):
        slide_link_list.append(base_url_merk+data_rte["deckid"][i]+"/"+slide_merk[data_rte['deckid'][i]])
    elif(data_rte['deckid'][i] in dev_id):
        slide_link_list.append(base_url_dev+data_rte["deckid"][i]+"/"+mapping_35[data_rte['deckid'][i]])
    else:
        print(i)

In [32]:
data_rte['slide_link'] = slide_link_list

In [36]:
data_rte['slide_link'][111]

'http://c-cube-dev.indegene.com/asset/5d11ce583f6c5432268f7a58/Chinese_3_2.png'

In [37]:
data_rte.to_csv("/home/ajeeth/Documents/VCC/keyMessage/Indication_model/med_comp/Merck_RTE_Slices_Indication_predicted.csv", index=False)

# iDA